In [20]:
import import_ipynb
from tifffile import TiffFile
from tiff_to_png import page_split
from google.cloud import vision
from google.cloud.vision import types
import io
from PIL import Image, ImageDraw
from enum import Enum
import os
import cv2
import datetime
import json

In [21]:
path = input ("Give file name: ")

Give file name: doc.tiff


In [23]:
page_split (path)

tif = TiffFile(path)
num = len (tif.pages)
print (num)

8


In [25]:

for i in range (num):
    
    image_file = path + "page%d.png" % i
    image  = Image.open(image_file)

    new_files = datetime.datetime.now()
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'token.json'
    client = vision.ImageAnnotatorClient()
    with io.open(image_file, 'rb') as image_file1:
        content = image_file1.read()
    content_image = types.Image(content=content)
    response = client.document_text_detection(image=content_image)
    document = response.full_text_annotation.text

    with open(str(new_files)+".docx", 'a') as f:
        print(document+"\n\n\n\n\n", file=f)
    
    os.remove(image_file)
